# Data extraction - Income tax

In [446]:
import time
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import requests as req
from bs4 import BeautifulSoup as bs 
import unicodedata
from functools import reduce


-----------------------------

## Beautiful Soup -  Spanish Tax Agency

In [447]:
url_scraping= "https://www.agenciatributaria.es/AEAT/Contenidos_Comunes/La_Agencia_Tributaria/Estadisticas/Publicaciones/sites/irpfmunicipios/2019/jrubikf74b3dca9af01b51cabd6d5603e0e16daecd1a97c.html"

In [448]:
html=req.get(url_scraping).content

soup=bs(html, 'html.parser')

tabla=soup.find_all('table')  # find_all(tag)

In [449]:
columnas=[e.text.strip() for e in tabla[0].find_all('th')]

#### Get namecols "RENTA BRUTA MEDIA , RENTA DISPONIBLE MEDIA'

In [450]:
cols = columnas[6:8]
cols

['RENTA BRUTA MEDIA', 'RENTA DISPONIBLEMEDIA']

In [451]:
cols_ccaa = [item.replace('RENTA BRUTA MEDIA','renta_bruta_media').replace('RENTA DISPONIBLEMEDIA', 'renta_disponible_media') for item in cols]
cols_ccaa.insert(0, "ccaa")

In [452]:
cols_provincias=[item.replace('RENTA BRUTA MEDIA','renta_bruta_media').replace('RENTA DISPONIBLEMEDIA', 'renta_disponible_media') for item in cols]
cols_provincias.insert(0, "provincia")

#### Get CCAA Data (except Pais Vasco and Navarra)

In [453]:
ccaa_renta_bruta_media = []
ccaa_renta_disponible_media = []
ccaa=[]

In [454]:
#Get data for CCAA
for table in soup.find_all("tr", class_="depth_1"):
    ccaa.append(table.find_all('th')[0].text)
    rb=table.find('td', {'headers': 'table01_c6'})
    ccaa_renta_bruta_media.append(rb.text)
    rm=table.find('td', {'headers': 'table01_c7'})
    ccaa_renta_disponible_media.append(rm.text)

In [455]:
#Erase characters
ccaa=[el.replace(u'\xa0','') for el in ccaa]
ccaa=[el.strip() for el in ccaa]

### Get data from provinces

In [456]:
provinces_table = soup.findAll(class_= 'depth_2') 

In [457]:
provincias_renta_bruta_media = []
provincias_renta_disponible_media = []
provincias=[]

In [458]:
for table in soup.find_all('tr', class_='depth_2'):
    provincias.append(table.find_all('th')[0].text)
    provincias_renta_bruta_media.append(table.find('td', {'headers': 'table01_c6'}).text)
    provincias_renta_disponible_media.append(table.find('td', {'headers': 'table01_c7'}).text)

In [459]:
#Erase characters
provincias=[el.replace(u'\xa0','') for el in provincias]
provincias=[el.strip() for el in provincias]

---------------------------

#### Get Pais Vasco data


In [460]:
url_PV = 'https://www.eustat.eus/elementos/ele0005700/renta-personal-media-de-la-c-a-de-euskadi-por-ambitos-territoriales-segun-tipo-de-renta-euros/tbl0005790_c.html'

In [461]:
html=req.get(url_PV).content

soup=bs(html, 'html.parser')

tabla_e=soup.find_all('table')

In [462]:
#rb=table.find_all('td', {'class': 'td8Bold'})[-1].text Coger el ultimo

In [463]:
for i,table in enumerate(soup.find_all("tr", class_="Nivel3B")):
    if i==0:
        ccaa.append(table.find('td').text)
        ccaa_renta_bruta_media.append(table.find('td', {'class': 'td8Bold'}).text)
        ccaa_renta_disponible_media.append(table.find_all('td', {'class': 'td8Bold'})[-1].text)

    elif i==1:
        provincias.append(table.find('td').text)
        provincias_renta_bruta_media.append(table.find('td', {'class': 'td8'}).text)
        provincias_renta_disponible_media.append(table.find_all('td', {'class': 'td8'})[-1].text)

    elif i==2:
        provincias.append(table.find('td').text)
        provincias_renta_bruta_media.append(table.find('td', {'class': 'td8'}).text)
        provincias_renta_disponible_media.append(table.find_all('td', {'class': 'td8'})[-1].text)
    
    elif i==3:
        provincias.append(table.find('td').text)
        provincias_renta_bruta_media.append(table.find('td', {'class': 'td8'}).text)
        provincias_renta_disponible_media.append(table.find_all('td', {'class': 'td8'})[-1].text)

In [464]:
ccaa[-1]='Pais Vasco'

In [465]:
ccaa=[el.replace(u'\xa0','') for el in ccaa]
ccaa=[el.strip() for el in ccaa]

In [466]:
provincias=[el.replace(u'\xa0','') for el in provincias]
provincias=[el.strip() for el in provincias]

---------------------------

#### Get Navarra data

In [467]:
ccaa.insert(20,'Navarra')
ccaa_renta_disponible_media.append(13.394)
ccaa_renta_bruta_media.append(16.300)

In [468]:
provincias.insert(53,'Navarra')
provincias_renta_bruta_media.append(13.394)
provincias_renta_disponible_media.append(16.300)

In [469]:
df_ccaa_renta = pd.DataFrame(zip(ccaa, ccaa_renta_bruta_media,ccaa_renta_disponible_media),
               columns =[cols_ccaa])

In [470]:
df_provincias_renta = pd.DataFrame(zip(provincias, provincias_renta_bruta_media,provincias_renta_disponible_media),
               columns =[cols_provincias])

In [479]:
df_ccaa_renta.to_csv('../data/ccaa_renta.csv', encoding='latin1', index=False)

In [480]:
df_provincias_renta.to_csv('../data/provincias_renta.csv', encoding='latin1', index=False)